# Project 2: Digit Recognition

## Statistical Machine Learning (COMP90051), Semester 2, 2017

*Copyright the University of Melbourne, 2017*

### Submitted by:  Yitong Chen
### Student number: 879326
### Kaggle-in-class username: *YitongChen*

In this project, you will be applying machine learning for recognising digits from real world images. The project worksheet is a combination of text, pre-implemented code and placeholders where we expect you to add your code and answers. You code should produce desired result within a reasonable amount of time. Please follow the instructions carefully, **write your code and give answers only where specifically asked**. In addition to worksheet completion, you are also expected to participate **live competition with other students in the class**. The competition will be run using an on-line platform called Kaggle.

** Marking:** You can get up to 33 marks for Project 2. The sum of marks for Project 1 and Project 2 is then capped to 50 marks

**Due date:** Wednesday 11/Oct/17, 11:59pm AEST (LMS components); and Kaggle competition closes Monday 09/Oct/17, 11:59pm AEST.

**Late submissions** will incur a 10% penalty per calendar day

** Submission materials**
 - **Worksheet**: Fill in your code and answers within this IPython Notebook worksheet.
 - **Competition**: Follow the instructions provided in the corresponding section of this worksheet. Your competition submissions should be made via Kaggle website.
 - **Report**: The report about your competition entry should be submitted to the LMS as a PDF file (see format requirements in `2.2`).
 - **Code**: The source code behind your competition entry.
The **Worksheet**, **Report** and **Code** should be bundled into a `.zip` file (not 7z, rar, tar, etc) and submitted in the LMS. Marks will be deducted for submitting files in other formats, or we may elect not to mark them at all.

**Academic Misconduct:** Your submission should contain only your own work and ideas. Where asked to write code, you cannot re-use someone else's code, and should write your own implementation. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

**Table of Contents**

1. Handwritten Digit Recognition **(16 marks)**
  1. Linear Approach
  2. Basis Expansion
  3. Kernel Perceptron
  4. Dimensionality Reduction
  
2. Kaggle Competition **(17 marks)**
  1. Making Submissions
  2. Method Description

## 1. Handwritten Digit Recognition
Handwritten digit recognition can be framed as a classification task: given a bitmap image as input, predict the digit type (0, 1, ..., 9). The pixel values in each position of the image form our features, and the digit type is the class. We are going to use a dataset where the digits are represented as *28 x 28* bitmap images. Each pixel value ranges between 0 and 1, and represents the monochrome ink intensity at that position. Each image matrix has been flattened into one long feature vector, by concatenating each row of pixels.

In this part of the project, we will only use images of two digits, namely "7" and "9". As such, we will be working on a binary classification problem. *Throughout this first section, our solution is going to be based on the perceptron classifier.*

Start by setting up working environment, and loading the dataset. *Do not override variable `digits`, as this will be used throughout this section.*

In [ ]:
%pylab inline

digits = np.loadtxt('digits_7_vs_9.csv', delimiter=' ')

Take some time to explore the dataset. Note that each image of "7" is labeled as -1, and each image of "9" is labeled as +1.

In [ ]:
# extract a stack of 28x28 bitmaps
X = digits[:, 0:784]
# extract labels for each bitmap
y = digits[:, 784:785]
# display a single bitmap and print its label
bitmap_index = 0
plt.imshow(X[bitmap_index,:].reshape(28, 28), interpolation=None)
print(y[bitmap_index])

You can also display several bitmaps at once using the following code.

In [ ]:
def gallery(array, ncols):
    nindex, height, width = array.shape
    nrows = nindex//ncols
    result = (array.reshape((nrows, ncols, height, width))
              .swapaxes(1,2)
              .reshape((height*nrows, width*ncols)))
    return result

ncols = 10
result = gallery(X.reshape((300, 28, 28))[:ncols**2], ncols)
plt.figure(figsize=(10,10))
plt.imshow(result, interpolation=None)

### 1.1 Linear Approach
We are going to use perceptron for our binary classification task. Recall that perceptron is a linear method. Also, for this first step, we will not apply non-linear transformations to the data.

Implement and fit a perceptron to the data above. You may use the implementation from *sklearn*, or implementation from one of our workshops. Report the error of the fit as the proportion of misclassified examples.

<br />

<font color='red'>**Write your code in the cell below ...**</font>

In [ ]:
## your code here
# To do this, simply concatenate a column of 1s to the data matrix.
Phi = np.column_stack([np.ones(X.shape[0]), X])
# Prediction function
def perc_pred(phi, w):
    return np.sign(np.sign(np.dot(phi, w)) + 0.5)

# Training algorithm
def train(data, target, epochs, w , eta= 1.):
    for e in range(epochs):
        for i in range(data.shape[0]):
            yhat = perc_pred(data[i,:], w)
            if yhat != target[i]:
                w += eta * target[i] * data[i]
    return w

# Run the training algorithm for 100 epochs to learn the weights
w = np.zeros(Phi.shape[1])
w = train(Phi, y, 100, w)
print(w.shape)

Error = np.sum(perc_pred(Phi, w).reshape(y.shape[0],1) != y) / float(y.shape[0])
print(Error)

One of the advantages of a linear approach is the ability to interpret results. To this end, plot the parameters learned above. Exclude the bias term if you were using it, set $w$ to be the learned perceptron weights, and run the following command.

In [ ]:
#print(w.reshape(1,785))
w = np.delete(w.reshape(1,785),0,1)
plt.imshow(w.reshape(28,28), interpolation=None)

In a few sentences, describe what you see, referencing which features are most important for making classification. Report any evidence of overfitting.

<font color='red'>**Write your answer here ...**</font> (as a *markdown* cell)

The importance of a feature is captured by computing how much the learned model depends on, as the epoch here.
The result is blurry, and the error proportion of misclassified examples is always 0.

Split the data into training and heldout validation partitions by holding out a random 25% sample of the data. Evaluate the error over the course of a training run, and plot the training and validation error rates as a function of the number of passes over the training dataset.

<br />
<font color='red'>**Write your code in the cell below ...**</font>

In [ ]:
## your code here
from sklearn.model_selection import train_test_split

phi_train, phi_test, y_train, y_test = train_test_split(Phi, y, 
                                                    test_size=0.25, 
                                                    random_state=0)

w_hat = np.zeros(Phi.shape[1])
T = 60
train_error = np.zeros(T)
heldout_error = np.zeros(T)
for ep in range(T):
    # here we use a learning rate, which decays with each epoch
    lr = 1./(1+ep)
    w_hat = train(Phi, y, 1, w_hat, eta = lr )
    #print(w_hat)
    train_error[ep] = np.sum(perc_pred(phi_train, w_hat).reshape(y_train.shape[0],1) != y_train) / np.float(y_train.shape[0])
    heldout_error[ep] = np.sum(perc_pred(phi_test, w_hat).reshape(y_test.shape[0],1) != y_test) / np.float(y_test.shape[0])

plot(train_error, label = 'Train Error')
plot(heldout_error, label = 'Held-out Error')
plt.legend()
xlabel('Epochs')
ylabel('Error')

In a few sentences, describe the shape of the curves, and compare the two. Now consider if we were to stop training early, can you choose a point such that you get the best classification performance? Justify your choice.

<font color='red'>**Write your answer here ...**</font> (as a *markdown* cell)

The shape was at a very high point at the beginning, and then went down sharply, but it increased a little at around 7 epoches, finally decreased to an even low level at around 9, so it is not good to stop training early. We may choose the point as 10, which is a reasonable low point for both train and holdout error rate.

Now that we have tried a simple approach, we are going to implement several non-linear approaches to our task. Note that we are still going to use a linear method (the perceptron), but combine this with a non-linear data transformation. We start with basis expansion.

### 1.2 Basis Expansion
Apply Radial Basis Function (RBF)-based transformation to the data, and fit a perceptron model. Recall that the RBF basis is defined as

$$\varphi_l(\mathbf{x}) =  \exp\left(-\frac{||\mathbf{x} - \mathbf{z}_l||^2}{\sigma^2}\right)$$

where $\mathbf{z}_l$ is centre of the $l^{th}$ RBF. We'll use $L$ RBFs, such that $\varphi(\mathbf{x})$ is a vector with $L$ elements. The spread parameter $\sigma$ will be the same for each RBF.

*Hint: You will need to choose the values for $\mathbf{z}_l$ and $\sigma$. If the input data were 1D, the centres $\mathbf{z}_l$ could be uniformly spaced on a line. However, here we have 784-dimensional input. For this reason you might want to use some of the training points as centres, e.g., $L$ randomly chosen "2"s and "7"s.*

<br />

<font color='red'>**Write your code in the cell below ...**</font>

In [ ]:
## your code here
#Each RBF basis function should take x (a 784 dimensional vector), and return a scalar. 
#Each RBF will be parameterised by a centre (784 dimensional vector) and a length scale (scalar). 
#The return scalar is computed based on the distance between x and the centre, as shown in the mathematical formulation.  
#Consequently phi(x) should return a vector of size L, and accordingly Phi(X) for the whole dataset will be a matrix N x L. 

# Input:
# x - is a column vector of input values
# z - is a scalar that controls location
# s - is a scalar that controls spread
#
# Output:
# v - contains the values of RBF evaluated for each element x
#     v has the same dimensionality as x
def radial_basis_function(x, z, s):
    # ensure that t is a column vector
    '''
    x = np.array(x)
    if x.size == 1:
        x.shape = (1,1)
    else:
        x_length = x.shape[0]
        x.shape = (x_length, 1)
    '''
    # compute RBF value
    r = np.linalg.norm(x - z)
    v = np.exp(-r**2/(s**2))
    return v

# Input:
# x - is an Nx784 column vector
# z - is an Lx784 column vector with locations for each of M RBFs
# s - is a scalar that controls spread, shared between all RBFs
#
# Output:
# Phi - is an NxL matrix, such that Phi(i,j) is the 
#       RBF transformation of x(i) using location z(j) and scale s
def expand_to_RBF(x, z, s):
    #... your code here ...
    #... in your code use "radial_basis_function" from above ...
    L = z.shape[0]
    N = x.shape[0]
    Phi = np.zeros((N, L))
    for i in range(N):
        for j in range(L):
        
            #y_rbf = radial_basis_function(x_rbf, z[i], sigma)
            v = radial_basis_function(x[i], z[j], sigma)
            Phi[i,j] = v
            
    
    return Phi

# set L to 60 and sigma to 0.01
l = 60
z = X[np.random.choice(X.shape[0], l, replace=False), :]
sigma = 0.01 # same scale for each RBF

# use "expand_to_RBF" function from above
x = expand_to_RBF(X, z, sigma)
print(x.shape)
x_dummy = np.ones(X.shape[0])
X_expand = np.column_stack((x_dummy, x))
print(X_expand.shape)

# Run the training algorithm for 100 epochs to learn the weights
w = np.zeros(X_expand.shape[1])
w = train(X_expand, y, 100, w)
print(w.shape)

Error = np.sum(perc_pred(X_expand, w).reshape(y.shape[0],1) != y) / float(y.shape[0])
print(Error)



Now compute the validation error for your RBF-perceptron and use this to choose good values of $L$ and $\sigma$. Show a plot of the effect of changing each of these parameters, and justify your parameter choice.

<br />

<font color='red'>**Write your code in the cell below ...**</font>

In [ ]:
## your code here
## when sigma is always 0.01
sigma = 0.01 # same scale for each RBF
train_error = np.zeros(300)
heldout_error = np.zeros(300)
for L in range(300):
    z = X[np.random.choice(X.shape[0], L, replace=False), :]
    x = expand_to_RBF(X, z, sigma)
    #print(x.shape)
    x_dummy = np.ones(X.shape[0])
    X_expand = np.column_stack((x_dummy, x))
    w_hat = np.zeros(X_expand.shape[1])
    w_hat = train(X_expand, y, 100, w_hat)
    
    phi_train, phi_test, y_train, y_test = train_test_split(X_expand, y, 
                                                    test_size=0.25, 
                                                    random_state=0)
    #print(w_hat)
    train_error[L] = np.sum(perc_pred(phi_train, w_hat).reshape(y_train.shape[0],1) != y_train) / np.float(y_train.shape[0])
    heldout_error[L] = np.sum(perc_pred(phi_test, w_hat).reshape(y_test.shape[0],1) != y_test) / np.float(y_test.shape[0])
plt.title('Errors for sigma = 0.01')
plot(train_error, label = 'Train Error')
plot(heldout_error, label = 'Held-out Error')
plt.legend()
xlabel('number of L')
ylabel('Error')

In [ ]:
# when L is always 300
l = 300
train_error_list = []
heldout_error_list = []
sigmas = [1e-10, 1e-6, 1e-4, 1e-2, 1, 100]
for n, s in enumerate(sigmas):
    z = X[np.random.choice(X.shape[0], l, replace=False), :]
    x = expand_to_RBF(X, z, s)
    #print(x.shape)
    x_dummy = np.ones(X.shape[0])
    X_expand = np.column_stack((x_dummy, x))
    w_hat = np.zeros(X_expand.shape[1])
    w_hat = train(X_expand, y, 100, w_hat)
    
    phi_train, phi_test, y_train, y_test = train_test_split(X_expand, y, 
                                                    test_size=0.25, 
                                                    random_state=0)
    #print(w_hat)
    train_error_list.append(np.sum(perc_pred(phi_train, w_hat).reshape(y_train.shape[0],1) != y_train) / np.float(y_train.shape[0]))
    heldout_error_list.append(np.sum(perc_pred(phi_test, w_hat).reshape(y_test.shape[0],1) != y_test) / np.float(y_test.shape[0]))
    
plt.title('Errors for L = 300')
plt.plot(sigmas[:6], np.asarray(train_error_list))
plt.plot(sigmas[:6], np.asarray(heldout_error_list))
plt.xlim((min(sigmas), max(sigmas)))
plt.xscale('log')
plt.ylim(0, 1)
#plot(heldout_error, label = 'Held-out Error')
xlabel('sigma')
ylabel('Error')

<font color='red'>**Write your justfication here ...**</font> (as a *markdown* cell)

From the first plot, we can just set sigma to 0.01 at first, and found that the error rate went down when L became close to 300, thus we set the L to 300 when testing for sigma, as in the second plot. 
From the second plot, it seems that the sigma has no effect on the performance.
As the result, we will just choose L as 300 with sigma 0.01.

### 1.3 Kernel Perceptron
Next, instead of directly computing a feature space transformation, we are going to use the kernel trick. Specifically, we are going to use the kernelised version of perceptron in combination with a few different kernels.

*In this section, you cannot use any libraries other than `numpy` and `matplotlib`.*

First, implement linear, polynomial and RBF kernels. The linear kernel is simply a dot product of its inputs, i.e., there is no feature space transformation. Polynomial and RBF kernels should be implemented as defined in the lecture slides.

<br />

<font color='red'>**Write your code in the cell below ...**</font>

In [ ]:
# Input:
# u,v - column vectors of the same dimensionality
#
# Output:
# v - a scalar
def linear_kernel(u, v):
    ## your code here
    z = np.dot(u.T, v)
    return z
# Input:
# u,v - column vectors of the same dimensionality
# c,d - scalar parameters of the kernel as defined in lecture slides
#
# Output:
# v - a scalar
def polynomial_kernel(u, v, c=0, d=3):
    ## your code here
    z = (np.dot(u.T, v)+c)**d
    return z   
   

# Input:
# u,v - column vectors of the same dimensionality
# gamma - scalar parameter of the kernel as defined in lecture slides
#
# Output:
# v - a scalar
def rbf_kernel(u, v, gamma=1):
    ## your code here
    r = np.linalg.norm(u - v)
    v = np.exp(-gamma*(r**2))
    return v

Kernel perceptron was a "green slides" topic, and you will not be asked about this method in the exam. Here, you are only asked to implement a simple prediction function following the provided equation. In kernel perceptron, the prediction for instance $\mathbf{x}$ is made based on the sign of

$$w_0 + \sum_{i=1}^{n}\alpha_i y_i K(\mathbf{x}_i, \mathbf{x})$$

Here $w_0$ is the bias term, $n$ is the number of training examples, $\alpha_i$ are learned weights, $\mathbf{x}_i$ and $y_i$ is the training dataset,and $K$ is the kernel.

<br />

<font color='red'>**Write your code in the cell below ...**</font>

In [ ]:
# Input:
# x_test - (r x m) matrix with instances for which to predict labels
# X - (n x m) matrix with training instances in rows
# y - (n x 1) vector with labels
# alpha - (n x 1) vector with learned weigths
# bias - scalar bias term
# kernel - a kernel function that follows the same prototype as each of the three kernels defined above
#
# Output:
# y_pred - (r x 1) vector of predicted labels

def kernel_ptron_predict(x_test, X, y, alpha, bias, kernel, c=0 ,d=3 ,gamma=1):
    ## your code here
    # test x_test is a matrix or vector
    if x_test.size == 784:
        R = 1
    else:
        R = x_test.shape[0]
    #R = int(x_test.shape[0]/784)
    #print(R)
    x_test = x_test.reshape(R,784)
    N = X.shape[0]
    y_pred = np.zeros((R,1))
    for i in range(R):
        for j in range(N):
            if kernel == linear_kernel:
                y_pred[i] += alpha[j]*y[j]*kernel(x_test[i],X[j])
            elif kernel == polynomial_kernel:
                y_pred[i] += alpha[j]*y[j]*kernel(x_test[i],X[j],c,d)
            else:
                y_pred[i] += alpha[j]*y[j]*kernel(x_test[i],X[j],gamma)
        y_pred[i] += bias
        y_pred[i]= np.sign(y_pred[i])
    return y_pred


The code for kernel perceptron training is provided below. You can treat this function as a black box, but we encourage you to understand the implementation.

In [ ]:
# Input:
# X - (n x m) matrix with training instances in rows
# y - (n x 1) vector with labels
# kernel - a kernel function that follows the same prototype as each of the three kernels defined above
# epochs - scalar, number of epochs
#
# Output:
# alpha - (n x 1) vector with learned weigths
# bias - scalar bias term
def kernel_ptron_train(X, y, kernel, epochs=100):
    n, m = X.shape
    alpha = np.zeros(n)
    bias = 0
    updates = None
    for epoch in range(epochs):
        print('epoch =', epoch, ', updates =', updates)
        updates = 0

        schedule = list(range(n))
        np.random.shuffle(schedule)
        for i in schedule:
            y_pred = kernel_ptron_predict(X[i], X, y, alpha, bias, kernel)
            
            if y_pred != y[i]:
                alpha[i] += 1
                bias += y[i]
                updates += 1

        if updates == 0:
            break
        
    return alpha, bias

Now use the above functions to train the perceptron. Use heldout validation, and compute the validation error for this method using each of the three kernels. Write a paragraph or two with analysis how the accuracy differs between the different kernels and choice of kernel parameters. Discuss the merits of a kernel approach versus direct basis expansion approach as was used in the previous section.

<br />

<font color='red'>**Write your code in the cell below ...**</font>

In [ ]:
phi_train, phi_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=0)
# Linear kernel
print('LINEAR')
alpha, bias = kernel_ptron_train(phi_train, y_train, linear_kernel)
# exclude bias term
#bias = 0
Error = np.sum(kernel_ptron_predict(phi_test, phi_train, y_train, alpha, bias, linear_kernel) != y_test) / float(y_test.shape[0])
print('Error =',Error)
#print(kernel_ptron_predict(phi_test, phi_train, y_train, alpha, bias, linear_kernel))

# Polynomial kernel
print('POLYNOMIAL')
alpha, bias = kernel_ptron_train(phi_train, y_train, polynomial_kernel)
# exclude bias term
#bias = 0
d_list = [0, 1, 10, 100]
for n, s in enumerate(d_list):
    Error = np.sum(kernel_ptron_predict(phi_test, phi_train, y_train, alpha, bias, polynomial_kernel, d = s) != y_test) / float(y_test.shape[0])
    print('d:',s,'Error =',Error)

c_list = [0, 1e-2, 1, 10, 100]
for n, s in enumerate(c_list):
    Error = np.sum(kernel_ptron_predict(phi_test, phi_train, y_train, alpha, bias, polynomial_kernel, c = s) != y_test) / float(y_test.shape[0])
    print('c:',s,'Error =',Error)
#print(kernel_ptron_predict(phi_test, phi_train, y_train, alpha, bias, polynomial_kernel))

# RBF kernel
print('RBF')
alpha, bias = kernel_ptron_train(phi_train, y_train, rbf_kernel)
# exclude bias term
#bias = 0
g_list = [ 1e-10, 1e-6, 1e-4, 1e-2, 1, 10, 100]
for n, s in enumerate(g_list):
    Error = np.sum(kernel_ptron_predict(phi_test, phi_train, y_train, alpha, bias, rbf_kernel, gamma = s) != y_test) / float(y_test.shape[0])
    print('g:',s,'Error =',Error)

#print(kernel_ptron_predict(phi_test, phi_train, y_train, alpha, bias, rbf_kernel))

<font color='red'>**Provide your analysis here ...**</font> (as a *markdown* cell)

It seems that the accuracy of the polynomial and rbf kernel are slightly better than the linear method, but they requir the good choice of parameters.
As for the polynomial kernel, it may have the best accuracy when c is 10 as well as d is 0-1.
As for the rbf kernel, it could have the best performance when gamma is 1.
The basis expansion relies on a large number of L, whereas the kernel approach is much easier to get a high accuracy.

### 1.4 Dimensionality Reduction
Yet another approach to working with complex data is to use a non-linear dimensionality reduction. To see how this might work, first apply a couple of dimensionality reduction methods and inspect the results.

In [ ]:
from sklearn import manifold
# extract a stack of 28x28 bitmaps
#X = digits[:, 0:784]
# extract labels for each bitmap
y = digits[:, 784:785]
#print(y)

X = digits[:, 0:784]
y = np.squeeze(digits[:, 784:785])

print(y)
# n_components refers to the number of dimensions after mapping
# n_neighbors is used for graph construction
X_iso = manifold.Isomap(n_neighbors=30, n_components=2).fit_transform(X)


# n_components refers to the number of dimensions after mapping
embedder = manifold.SpectralEmbedding(n_components=2, random_state=0)
X_se = embedder.fit_transform(X)

f, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(X_iso[y==-1,0], X_iso[y==-1,1], "bo")
ax1.plot(X_iso[y==1,0], X_iso[y==1,1], "ro")
ax1.set_title('Isomap')
ax2.plot(X_se[y==-1,0], X_se[y==-1,1], "bo")
ax2.plot(X_se[y==1,0], X_se[y==1,1], "ro")
ax2.set_title('spectral')

In a few sentences, explain how a dimensionality reduction algorithm can be used for your binary classification task.

<font color='red'>**Write your answer here ...**</font> (as a *markdown* cell)

There are two ﬁelds in dimension reduction, linear techniques that use a linear mapping to reduce the dimension, and nonlinear technique makes the assumption that the data available is embedded on a manifold (or surface in lower dimensional space). The data is then mapped onto a lower-dimensional manifold for more eﬃcient processing.


Implement such an approach and assess the result. For simplicity, we will assume that both training and test data are available ahead of time, and thus the datasets should be used together for dimensionality reduction, after which you can split off a test set for measuring generalisation error. *Hint: you do not have to reduce number of dimensions to two. You are welcome to use the sklearn library for this question.*
 
<br />

<font color='red'>**Write your code in the cell below ...**</font>

In [ ]:
from sklearn.linear_model import perceptron

# n_components refers to the number of dimensions after mapping
# n_neighbors is used for graph construction
X_iso = manifold.Isomap(n_neighbors=30, n_components=2).fit_transform(X)

# split off the test data
phi_train, phi_test, y_train, y_test = train_test_split(X_iso, y, 
                                                    test_size=0.25, 
                                                    random_state=0)

# Create the model
clf = perceptron.Perceptron()
clf.fit(phi_train, y_train)

Error = np.sum(clf.predict(phi_test) != y_test) / float(y_test.shape[0])
print(Error)

# Create the KMeans model
#clf = cluster.KMeans(init='k-means++', n_clusters=10, random_state=42)

# Fit the training data to the model
#clf.fit(X_train)

f, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(X_iso[np.where(clf.predict(phi_test)==-1),0], X_iso[np.where(clf.predict(phi_test)==-1),1], "bo")
ax1.plot(X_iso[np.where(clf.predict(phi_test)==1),0], X_iso[np.where(clf.predict(phi_test)==1),1], "ro")
ax1.set_title('Predicted Training Labels for spectral')
ax2.plot(X_iso[np.where(y_test==-1),0], X_iso[np.where(y_test==-1),1], "bo")
ax2.plot(X_iso[np.where(y_test==1),0], X_iso[np.where(y_test==1),1], "ro")
ax2.set_title('Actual Training Labels for Isomap')


In [ ]:
# split off the test data
phi_train, phi_test, y_train, y_test = train_test_split(X_se, y, 
                                                    test_size=0.25, 
                                                    random_state=0)

# Create the model
clf = perceptron.Perceptron()
clf.fit(phi_train, y_train)

Error = np.sum(clf.predict(phi_test) != y_test) / float(y_test.shape[0])
print(Error)

# Create the KMeans model
#clf = cluster.KMeans(init='k-means++', n_clusters=10, random_state=42)

# Fit the training data to the model
#clf.fit(X_train)

f, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(X_se[np.where(clf.predict(phi_test)==-1),0], X_se[np.where(clf.predict(phi_test)==-1),1], "bo")
ax1.plot(X_se[np.where(clf.predict(phi_test)==1),0], X_se[np.where(clf.predict(phi_test)==1),1], "ro")
ax1.set_title('Predicted Training Labels for spectral')
ax2.plot(X_se[np.where(y_test==-1),0], X_se[np.where(y_test==-1),1], "bo")
ax2.plot(X_se[np.where(y_test==1),0], X_se[np.where(y_test==1),1], "ro")
ax2.set_title('Actual Training Labels for spectral')

In a few sentences, comment on the merits of the dimensionality reduction based approach compared to linear classification from Section 1.1 and basis expansion from Section 1.2.

<font color='red'>**Write your answer here ...**</font> (as a *markdown* cell)

From the error rate and the plots, we can find the performance of dimensionality reduction approach is pretty good, especially from the plots, which make the results clear and intuitive.

The merits over the linear classification is the classification becoming obviously faster, as well as getting high accuracy easily, which is because:
1. It reduces the time and storage space required.
2. Removal of multi-collinearity improves the performance of the machine learning model.

Besides, it becomes easier to visualize the data when reduced to very low dimensions such as 2D or 3D, which could be more intuitive to look at the performance.

## 2. Kaggle Competition
The final part of the project is a competition, on more challenging digit data sourced from natural scenes. This data is coloured, pixelated or otherwise blurry, and the digits are not perfectly centered. It is often difficult for humans to classify! The dataset is also considerably larger. 

Please sign up to the [COMP90051 Kaggle competition](https://inclass.kaggle.com/c/comp90051-2017) using your `student.unimelb.edu.au` email address. Then download the file `data.npz` from Kaggle. This is a compressed `numpy` data file containing three ndarray objects:
 - `train_X` training set, with 4096 input features (greyscale pixel values);
 - `train_Y` training labels (0-9)
 - `test_X` test set, with 4096 input features, as per above
 
Each image is 64x64 pixels in size, which has been flattened into a vector of 4096 values. You should load the files using `np.load`, from which you can extract the three elements. You may need to transpose the images for display, as they were flattened in a different order. Each pixel has an intensity value between 0-255. For those using languages other than python, you may need to output these objects in another format, e.g., as a matlab matrix.

Your job is to develop a *multiclass* classifier on this dataset. You can use whatever techniques you like, such as the perceptron code from above, or other methods such as *k*NN, logistic regression, neural networks, etc. You may want to compare several methods, or try an ensemble combination of systems. You are free to use any python libraries for this question. Note that some fancy machine learning algorithms can take several hours or days to train (we impose no time limits), so please start early to allow sufficient time. *Note that you may want to sample smaller training sets, if runtime is an issue, however this will degrade your accuracy. Sub-sampling is a sensible strategy when developing your code.*

You may also want to do some basic image processing, however, as this is not part of the subject, we would suggest that you focus most of your efforts on the machine learning. For inspiration, please see [Yan Lecun's MNIST page](http://yann.lecun.com/exdb/mnist/), specifically the table of results and the listed papers. Note that your dataset is harder than MNIST, so your mileage may vary.

In [ ]:
%pylab inline
# load the files
train_X = np.load('data/train_X.npy')
train_y = np.load('data/train_y.npy')
test_X = np.load('data/test_X.npy')
print(train_X.shape)
print(train_X[0].shape)

#plt.imshow(train_X[0,:].reshape(64, 64), interpolation=None)

plt.subplot(221)
plt.imshow(train_X[0].reshape(64,64), cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(train_X[1].reshape(64,64), cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(train_X[2].reshape(64,64), cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(train_X[3].reshape(64,64), cmap=plt.get_cmap('gray'))


In [ ]:
%pylab inline
# Import `train_test_split`
from sklearn.model_selection import train_test_split
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

# load the files
train_X = np.load('data/train_X.npy')
train_y = np.load('data/train_y.npy')
test_X = np.load('data/test_X.npy')

# We can also reduce our memory requirements 
# by forcing the precision of the pixel values to be 32 bit
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

# normalize inputs from 0-255 to 0-1
train_X = train_X / 255
test_X = test_X / 255

# one hot encode outputs
#train_y = np_utils.to_categorical(train_y)

# Split the data into training and test sets 
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.25, random_state=42)

#----------------------------SVC--------------------------------
# Import the `svm` model
from sklearn import svm

# Create the SVC model 
svc_model = svm.SVC(gamma=0.001, C=100., kernel='linear')

# Fit the data to the SVC model
svc_model.fit(X_train, y_train)

print(svc_model.score(X_test, y_test))

#----------------------SVM candidates----------------------------
# Set the parameter candidates
parameter_candidates = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

# Create a classifier with the parameter candidates
clf = GridSearchCV(estimator=svm.SVC(), param_grid=parameter_candidates, n_jobs=1)

# Train the classifier on training data
clf.fit(X_train, y_train)

# Print out the results 
print('Best score for training data:', clf.best_score_)
print('Best `C`:',clf.best_estimator_.C)
print('Best kernel:',clf.best_estimator_.kernel)
print('Best `gamma`:',clf.best_estimator_.gamma)

# Apply the classifier to the test data, and view the accuracy score
print(clf.score(X_test, y_test))




In [ ]:
#--------------------------Baseline Model with Multi-Layer Perceptrons------------------------
%pylab inline
# Import `train_test_split`
from sklearn.model_selection import train_test_split
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

# load the files
train_X = np.load('data/train_X.npy')
train_y = np.load('data/train_y.npy')
test_X = np.load('data/test_X.npy')
print(train_X.shape)

# We can also reduce our memory requirements 
# by forcing the precision of the pixel values to be 32 bit
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

# normalize inputs from 0-255 to 0-1
train_X = train_X / 255
test_X = test_X / 255

# one hot encode outputs
train_y = np_utils.to_categorical(train_y)

# Split the data into training and test sets 
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.25, random_state=42)

num_pixels = X_train.shape[1]
num_classes = y_train.shape[1]

# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

print('end')

In [ ]:
#--------------------------Simple Convolutional Neural Network--------------------------
%pylab inline
# Import `train_test_split`
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load the files
train_X = np.load('data/train_X.npy')
train_y = np.load('data/train_y.npy')
test_X = np.load('data/test_X.npy')

# reshape to be [samples][pixels][width][height]
train_X = train_X.reshape(train_X.shape[0], 1, 64, 64).astype('float32')
test_X = test_X.reshape(test_X.shape[0], 1, 64, 64).astype('float32')

# normalize inputs from 0-255 to 0-1
train_X = train_X / 255
test_X = test_X / 255
# one hot encode outputs
train_y = np_utils.to_categorical(train_y)
num_classes = train_y.shape[1]

# Split the data into training and test sets 
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.25, random_state=42)

def baseline_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(1, 64, 64), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=400, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

np.savetxt("simple.csv",model.predict(test_X).argmax(1),delimiter=",")

In [ ]:
#-------------------Larger Convolutional Neural Network-------------------
%pylab inline
# Import `train_test_split`
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.callbacks import ModelCheckpoint

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load the files
train_X = np.load('data/train_X.npy')
train_y = np.load('data/train_y.npy')
test_X = np.load('data/test_X.npy')

# reshape to be [samples][pixels][width][height]
train_X = train_X.reshape(train_X.shape[0], 1, 64, 64).astype('float32')
test_X = test_X.reshape(test_X.shape[0], 1, 64, 64).astype('float32')

# normalize inputs from 0-255 to 0-1
train_X = train_X / 255
test_X = test_X / 255
# one hot encode outputs
train_y = np_utils.to_categorical(train_y)
num_classes = train_y.shape[1]

# Split the data into training and test sets 
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.25, random_state=42)

# define the larger model
def larger_model():
    # create model
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(1, 64, 64), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# build the model
model = larger_model()

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

np.savetxt("large_2.csv",model.predict(test_X).argmax(1),delimiter=",")


In [ ]:
#-------------------------------Visualization of results---------------------------------
from sklearn import manifold
# Create an isomap and fit the `digits` data to it
X_test_reshape = X_test.reshape(X_test.shape[0], 4096)

X_iso = manifold.Isomap(n_neighbors=10).fit_transform(X_test_reshape[0:1000])
print('prediction begins')
# Compute cluster centers and predict cluster index for each sample
predicted = model.predict(X_test[0:1000]).argmax(1)
print('plot')
# Create a plot with subplots in a grid of 1X2
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

# Adjust the layout
fig.subplots_adjust(top=0.85)

y_test_color = y_test[0:1000].argmax(1)

# Add scatterplots to the subplots 
ax[0].scatter(X_iso[:, 0], X_iso[:, 1], c=predicted)
ax[0].set_title('Predicted labels')
ax[1].scatter(X_iso[:, 0], X_iso[:, 1], c=y_test_color)
ax[1].set_title('Actual Labels')


# Add title
fig.suptitle('Predicted versus actual labels', fontsize=14, fontweight='bold')

# Show the plot
plt.show()

In [ ]:
#--------------------------Increase batch size-----------------------------------
# build the model
model = larger_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=2000)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

np.savetxt("large_3.csv",model.predict(test_X).argmax(1),delimiter=",")


In [ ]:
#----------------------------Increased layer larger CNN-------------------------------
%pylab inline
# Import `train_test_split`
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.callbacks import ModelCheckpoint

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load the files
train_X = np.load('data/train_X.npy')
train_y = np.load('data/train_y.npy')
test_X = np.load('data/test_X.npy')

# reshape to be [samples][pixels][width][height]
train_X = train_X.reshape(train_X.shape[0], 1, 64, 64).astype('float32')
test_X = test_X.reshape(test_X.shape[0], 1, 64, 64).astype('float32')

# normalize inputs from 0-255 to 0-1
train_X = train_X / 255
test_X = test_X / 255
# one hot encode outputs
train_y = np_utils.to_categorical(train_y)
num_classes = train_y.shape[1]

# Split the data into training and test sets 
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.25, random_state=42)

# define the larger model
def larger_model():
    # create model
    model = Sequential()
    model.add(Conv2D(64, (10, 10), input_shape=(1, 64, 64), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# build the model
model = larger_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

np.savetxt("large_4.csv",model.predict(test_X).argmax(1),delimiter=",")


### 2.1 Making Submissions

This will be setup as a *Kaggle in class* competition, in which you can upload your system predictions on the test set. You should format your predictions as a csv file, with the same number of lines as the test set, and each line comprising two numbers `id, class` where *id* is the instance number (increasing integers starting from 1) and *class* is an integer between 0-9, corresponding to your system prediction. E.g., 
```
Id,Label
1,9
2,9
3,4
4,5
5,1
...```
based on the first five predictions of the system being classes `9 9 4 5 1`. See the `sample_submission.csv` for an example file.

Kaggle will report your accuracy on a public portion of the test set, and maintain a leaderboard showing the performance of you and your classmates. You will be allowed to upload up to four submissions each day. At the end of the competition, you should nominate your best submission, which will be scored on the private portion of the test set. The accuracy of your system (i.e., proportion of correctly classified examples) on the private test set will be used for grading your approach.

**Marks will be assigned as follows**:
 - position in the class, where all students are ranked and then the ranks are linearly scaled to <br>0 marks (worst in class) - 4 marks (best in class) 
 - absolute performance (4 marks), banded as follows (rounded to nearest integer): 
 <br>below 80% = 0 marks; 80-89% = 1; 90-92% = 2; 93-94% = 3; above 95% = 4 marks

Note that you are required to submit your code with this notebook, submitted to the LMS. Failure to provide your implementation may result in assigning zero marks for the competition part, irrespective of the competition standing. Your implementation should be able to exactly reproduce submitted final Kaggle entry, and match your description below.

### 2.2. Method Description
Describe your approach, and justify each of the choices made within your approach. You should write a document with no more than 400 words, as a **PDF** file (not *docx* etc) with up to 2 pages of A4 (2 sides). Text must only appear on the first page, while the second page is for *figures and tables only*. Please use a font size of 11pt or higher. Please consider using `pdflatex` for the report, as it's considerably better for this purpose than wysiwyg document editors. You are encouraged to include empirical results, e.g., a table of results, graphs, or other figures to support your argument. *(this will contribute 9 marks; note that we are looking for clear presentation, sound reasoning, good evaluation and error analysis, as well as general ambition of approach.)*